# 1. [Wildcard Matching](https://leetcode.com/problems/wildcard-matching/)
- Give 2 strings s and p. Find if s and p can be matched
    + s could be empty and contains only lowercase letters a-z.
    + p could be empty and contains only lowercase letters a-z, and characters like ? or *.
    
- Rules
```
'?' Matches any single character.
'*' Matches zero or multiple characters.
    Eg: 
        * = ''
        * = a
        * = ab
        * = abcdefhgh
```

#### Examples

```
Input:
    s = "aa"
    p = "a"
Output: false

Input:
    s = "aa"
    p = "*"
Output: true

Input:
    s = "cb"
    p = "?a"
Output: false

Input:
    s = "adceb"
    p = "*a*b"
Output: true

Input:
    s = "acdcb"
    p = "a*c?b"
Output: false
```

## Solution 1
- Naive bottom up DP - O($n^3$)

```C++
class Solution {
public:
    bool isMatch(string &S, string &P) {
        int n = S.size();
        int m = P.size();

        // dp[i+1][j+1] marks the optimal solution for S[:i], P[:j]
        vector<vector<bool>> dp(n+1, vector(m+1, false));
        dp[0][0] = true;

        for(int i=0; i<=n; ++i) for(int j=0; j<=m; ++j) {
            // Propagate true only
            if(dp[i][j] == false) continue;

            // Case: normal matching
            if(i<n && j<m &&
                    (S[i] == P[j] || P[j] == '?')) {
                dp[i+1][j+1] = true;
            }

            // Case: P[j] == *
            else if(j<m && P[j] == '*') {
                // match S[i:] = P[j]
                int i_ = i;
                while(i_ <= n) dp[i_++][j+1] = true;
            }
        }

        return dp[n][m];
    }
};
```

## Solution 2
- Recursive bottom up DP - $O(n^2)$

```Cpp
class Solution {
private:
    string _S, _P;
    int _n, _m;

    vector<vector<bool>> _dp;

public:
    bool get(int i, int j) {
        if(i == _n && j == _m) return true;
            // matched
        if(i == _n) return (_P[j] == '*' && get(i, j+1));
            // Case S end and P[j] == '*'
            //      search for the next character of P[j+1] to confirm. Eg
            //          a == a*
            //          a != a*c
        if(j == _m) return false;
            // Unmatched

        if(_dp[i][j] != NULL) return _dp[i][j];
            // Get mem

        if(_P[j] == '*') return _dp[i][j] = (get(i+1, j) || get(i, j+1));
            // Case: P[j] == '*', S skip 1 character or P ignore '*' (* matchs 0 to inf)

        if(_P[j] == '?' || _S[i] == _P[j]) return _dp[i][j] = get(i+1, j+1);
            // Case: normal matching

        return false;
    }
    bool isMatch(string &S, string &P) {
        _S = S;
        _n = S.size();

        _P = P;
        _m = P.size();

        // dp[i+1][j+1] marks the optimal solution for S[:i], P[:j]
        _dp.assign(_n+1, vector<bool>(_m+1, NULL));

        // Left -> Right
        return get(0, 0);
    }
};
```

# 2. [Regular Expression Matching](https://leetcode.com/problems/regular-expression-matching/)
- Give 2 strings s and p. Find if s and p can be matched
    + s could be empty and contains only lowercase letters a-z.
    + p could be empty and contains only lowercase letters a-z, and characters like . or *.
    
```
'.' Matches any single character.
'*' Matches zero or more of the preceding element.
    Eg: 'c*' can be
        + ''
        + 'c'
        + 'cc'
        + 'ccc...c'
```
#### Examples

```
Input:
    s = "aa"
    p = "a"
Output: false

Input:
    s = "aa"
    p = "a*"
Output: true

Input:
    s = "ab"
    p = ".*"
Output: true

Input:
    s = "aab"
    p = "c*a*b"
Output: true

Input:
    s = "mississippi"
    p = "mis*is*p*."
Output: false

Input:
    s = "a"
    p = "ab*"
Output: false
```

## Solution 1
- DP bottom up - O($n^2$)

```C++
class Solution {
public:
    bool isMatch(string &S, string &P) {
        int n = S.size();
        int m = P.size();

        // dp[i+1][j+1] marks the optimal solution for S[:i], P[:j]
        vector<vector<bool>> dp(n+1, vector<bool>(m+1, false));
        dp[0][0] = true;

        for(int i=0; i<=n; ++i) for(int j=0; j<=m; ++j) {
            // Propagate true only
            if(dp[i][j] == false) continue;

             // Case: single char matching
            if(i<n && j<m &&
                    (S[i] == P[j] || P[j] == '.')) {
                dp[i+1][j+1] = true;
            }

            // Case: *
            if(j+1<m &&
                    P[j+1] == '*') {
                // P[j:j+2] = c* == ''
                dp[i][j+2] = true;

                // Consider P[j:j+2] = c*
                if(i<n &&
                        (P[j] == S[i] || P[j] == '.') && P[j+1] == '*') {
                    // c* == c
                    dp[i+1][j+2] = true;

                    // c* == c...c
                    dp[i+1][j] = true;
                }
            }

        }
        return dp[n][m];
    }
};
```

## Solution 2
- Recursive bottom up - O($n^2$)

```Cpp
class Solution {
private:
    string _S, _P;
    int _n, _m;

    vector<vector<bool>> _dp;
public:
    bool get(int i, int j) {
        if(i == _n && j == _m) return true;
        if(i > _n || j > _m) return false;
        if(_dp[i][j] != NULL) return _dp[i][j];
        bool case0 = false, case1 = false, case2 = false, case3 = false;

        // Case: single char matching
        if(_P[j] == '.' || _P[j] == _S[i]) {
            case0 = get(i+1, j+1);
        }

        // Consider P[j:j+2] = c*
        if(j+1 < _m && _P[j+1] == '*') {
            // c* = ''
            case1 = get(i, j+2);

            // c* = c
            case2 = (_P[j] == _S[i] || _P[j] == '.') && get(i+1, j+2);

            // c* = c...c
            case3 = (_P[j] == _S[i] || _P[j] == '.') && get(i+1, j);
        }

        return _dp[i][j] = (case0 || case1 || case2 || case3);
    }

    bool isMatch(string &S, string &P) {
        _S = S;
        _n = S.size();

        _P = P;
        _m = _P.size();

        _dp.assign(_n+1, vector<bool>(_m+1, NULL));

        // Left -> Right
        return get(0, 0);
    }
};
```